#### ETL Processes

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from array import *
import numpy as np

Created connection to the sparkifydb database and a cursor to it.

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

Function to fetch all files matching .json extension from directory

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

### Processing `song_data`
Performing ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

- Using the `get_files` function provided above to get a list of all song JSON files in `data/song_data`.

In [4]:
song_files = get_files("data/song_data")

Selecting the first song file in this list.

In [5]:
filepath = song_files[0]

Reading the song file and storing it in DataFrame called `df`

In [6]:
df = pd.read_json(filepath,lines = True)
df.head()

artist_id  artist_latitude artist_location  artist_longitude  \
0  ARNNKDK1187B98BBD5         45.80726  Zagreb Croatia           15.9676   

  artist_name   duration  num_songs             song_id  \
0        Jinx  407.37914          1  SOFNOQK12AB01840FC   

                         title  year  
0  Kutt Free (DJ Volume Remix)     0

## #1: `songs` Table
#### Extracting Data for Songs Table
- Selected columns for song ID, title, artist ID, year, and duration
- Dropped records with null values
- Dropped duplicates based on column `song_id`
- Used `df_songs.values` to select just the values from the dataframe `df_songs` to save in array called `arr_songs`
- Converted the array to a list called `song_data`
- Indexed to select the first (only) record in the list `song_data`

In [7]:
df_songs = df[['song_id', 'title','artist_id','year','duration']]
df_songs = df_songs.dropna()
df_songs = df_songs.drop_duplicates(subset='song_id', keep="last", inplace=False)
arr_songs = df_songs.values
song_data = np.array(arr_songs).tolist()
song_data[0]

['SOFNOQK12AB01840FC',
 'Kutt Free (DJ Volume Remix)',
 'ARNNKDK1187B98BBD5',
 0,
 407.37914]

#### Inserting Record into Song Table
By implementing the `song_table_insert` query in `sql_queries.py` to insert a record for this song into the `songs` table.

In [8]:
cur.execute(song_table_insert, song_data[0])
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extracting Data for Artists Table
- Selecting columns for artist ID, name, location, latitude, and longitude
- Dropped records with null values
- Dropped duplicates based on column `artist_id`
- Used `df_artists.values` to select just the values from the dataframe df_artists
- Converted the array to a list called artist_data
- Indexed to select the first (only) record in the list artist_data

In [9]:
df_artists = df[['artist_id', 'artist_name','artist_location','artist_latitude','artist_longitude']]
df_artists = df_artists.dropna()
df_artists = df_artists.drop_duplicates(subset='artist_id', keep="last", inplace=False)
arr_artists = df_artists.values
artist_data = np.array(arr_artists).tolist()
artist_data[0]

['ARNNKDK1187B98BBD5', 'Jinx', 'Zagreb Croatia', 45.80726, 15.967600000000001]

#### Inserting Record into Artist Table
Implementing the `artist_table_insert` query in `sql_queries.py` to insert a record for this song's artist into the `artists` table. 
- Also checking the length of artist_data list in case the list is empty the insert query shouldn't be executed otherwise we'll get an out of index error

In [10]:
if (len(artist_data) > 0):
    cur.execute(artist_table_insert, artist_data[0])
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Processing `log_data`
Performing ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Used the `get_files` function provided above to get a list of all log JSON files in `data/log_data`

In [11]:
log_files = get_files("data/log_data")

Selecting the first log file in the list `log_files`

In [12]:
filepath = log_files[0]

Reading the log file, storing in a DataFrame called df and dropping records with null values.

In [13]:
df = pd.read_json(filepath,lines = True)
df = df.dropna()
df.head()

artist       auth firstName gender  itemInSession lastName  \
0  Sydney Youngblood  Logged In     Jacob      M             53    Klein   
1         Gang Starr  Logged In     Layla      F             88  Griffin   
2              3OH!3  Logged In     Layla      F             89  Griffin   
3        RÃÂ¶yksopp  Logged In     Jacob      M             54    Klein   
4         Kajagoogoo  Logged In     Layla      F             90  Griffin   

      length level                             location method      page  \
0  238.07955  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
1  151.92771  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   
2  192.52200  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   
3  369.81506  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
4  223.55546  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   

   registration  sessionId                                         song  \
0  1.540558e+12        954                            Ain't No Sunshine   
1  1.541057e+12        984                   My Advice 2 You (Explicit)   
2  1.541057e+12        984  My First Kiss (Feat. Ke$ha) [Album Version]   
3  1.540558e+12        954                       The Girl and The Robot   
4  1.541057e+12        984                                      Too Shy   

   status             ts                                          userAgent  \
0     200  1543449657796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
1     200  1543449690796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
2     200  1543449841796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
3     200  1543449895796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
4     200  1543450033796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   

  userId  
0     73  
1     24  
2     24  
3     73  
4     24

## #3: `time` Table
#### Extracting Data for Time Table
- Filtered records by `NextSong` action

In [14]:
df_filtered = df[(df.page == 'NextSong')]
df_filtered.head()

artist       auth firstName gender  itemInSession lastName  \
0  Sydney Youngblood  Logged In     Jacob      M             53    Klein   
1         Gang Starr  Logged In     Layla      F             88  Griffin   
2              3OH!3  Logged In     Layla      F             89  Griffin   
3        RÃÂ¶yksopp  Logged In     Jacob      M             54    Klein   
4         Kajagoogoo  Logged In     Layla      F             90  Griffin   

      length level                             location method      page  \
0  238.07955  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
1  151.92771  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   
2  192.52200  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   
3  369.81506  paid  Tampa-St. Petersburg-Clearwater, FL    PUT  NextSong   
4  223.55546  paid         Lake Havasu City-Kingman, AZ    PUT  NextSong   

   registration  sessionId                                         song  \
0  1.540558e+12        954                            Ain't No Sunshine   
1  1.541057e+12        984                   My Advice 2 You (Explicit)   
2  1.541057e+12        984  My First Kiss (Feat. Ke$ha) [Album Version]   
3  1.540558e+12        954                       The Girl and The Robot   
4  1.541057e+12        984                                      Too Shy   

   status             ts                                          userAgent  \
0     200  1543449657796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
1     200  1543449690796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
2     200  1543449841796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   
3     200  1543449895796  "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...   
4     200  1543450033796  "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...   

  userId  
0     73  
1     24  
2     24  
3     73  
4     24

Converted the `ts` timestamp column to datetime

In [15]:
t = pd.to_datetime(df_filtered['ts'], unit='ms')
t.head()

0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

- Extracted the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and stored in a list `time_data` containing these values in order using pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access datetimelike properties.
- Specified labels for these columns and set to `column_labels`

In [16]:
time_data = [t.dt.time,t.dt.hour,t.dt.day,t.dt.week,t.dt.month,t.dt.year,t.dt.weekday]
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']

- Created a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe.
- Dropped records with null values
- Dropped duplciates on `start_time` since it's the primary key and can't contain multiple rows with same value

In [17]:
t_dict = dict(zip(column_labels,time_data))
time_df = pd.DataFrame.from_dict(t_dict)
time_df = time_df.dropna()
time_df = time_df.drop_duplicates(subset='start_time', keep="last", inplace=False)
time_df.head()

start_time  hour  day  week  month  year  weekday
0  00:00:57.796000     0   29    48     11  2018        3
1  00:01:30.796000     0   29    48     11  2018        3
2  00:04:01.796000     0   29    48     11  2018        3
3  00:04:55.796000     0   29    48     11  2018        3
4  00:07:13.796000     0   29    48     11  2018        3

#### Inserting Records into Time Table
Implemented the `time_table_insert` query in `sql_queries.py` to insert records for the timestamps in this log file into the `time` table.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extracting Data for Users Table
- Selected columns for user ID, first name, last name, gender and level and set to `user_df`
- Dropped records with null values using df.dropna()

In [19]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.dropna()
#user_df = user_df.drop_duplicates(subset='userId', keep="last", inplace=False)
user_df.head()

userId firstName lastName gender level
0     73     Jacob    Klein      M  paid
1     24     Layla  Griffin      F  paid
2     24     Layla  Griffin      F  paid
3     73     Jacob    Klein      M  paid
4     24     Layla  Griffin      F  paid

#### Inserting Records into Users Table
Implemented the `user_table_insert` query in `sql_queries.py` to insert records for the users in this log file into the `users` table. 

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extracting Data and Songplays Table
- Implemented the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Stored the results from `song_select` query in `results` variable.
- Converted the `ts` timestamp column to datetime
- Selected the datetime, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Inserting Records into Songplays Table
- Implemented the `songplay_table_insert` query to insert records for the songplay actions in this log file into the `songplays` table. 

In [22]:
for index, row in df.iterrows():
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        #insert songplay record
        t1 = pd.to_datetime(row.ts, unit='ms')
   
        songplay_data = (t1, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)

Run `test.ipynb` to see if you've successfully added records to this table.

# Closed Connection to Sparkify Database

In [23]:
conn.close()